<img src='http://www-scf.usc.edu/~ghasemig/images/sharif.png' alt="SUT logo" width=300 height=300 align=left class="saturate" >

<br>
<font>
<div dir=ltr align=center>
<font color=0F5298 size=7>
    Artificial Intelligence <br>
<font color=2565AE size=5>
    Computer Engineering Department <br>
    Spring 2023<br>
<font color=3C99D size=5>
    Practical Assignment 2 - CSP <br>
<font color=696880 size=4>
    Yalda Shabanzadeh, Ali Salesi

____

# Personal Data

In [1]:
# Set your student number
student_number = '99101087'
name = 'AmirReza'
last_name = 'Azari'

# Rules
- Make sure that all of your cells can be run perfectly. 

# Q1: Cryptarithmetic Puzzle (100 Points)

<font size=4>
Author: Yalda Shabanzadeh, Ali Salesi
<br/>
<font color=red>
Please run all the cells.
</font>
</font>
<br/>
</div>

## Problem Definition

The Cryptarithmetic Puzzle is a puzzle where the digits of some numbers are replaced with letters to make a mathematical equation. Each letter represents a unique digit from 0 to 9 (in case of base 10). The puzzle has the following constraints:

- Each letter can be replaced by a digit from 0 to 9 (in case of base 10), but no digit can be used twice.
- Each letter must be replaced by the same digit throughout the puzzle.
- The leftmost digit of each number cannot be zero.

### Example
Here's an example puzzle:

```
  two
+ two
------
 four
```

In this puzzle, we need to replace each letter with a digit from 0 to 9 (in case of base 10) to make the equation true.

## Note
- You can use any arbitrary CSP algorithm to solve this question.
- Allowed algorithms:
BackTracking, AC3, AC3-LCV, AC3-MRV, AC3-LCV-MRV

> Explain how you consider this problem as a CSP (5 points)


<font color='cyan'>
Your explanation.
</font><br>
<font  color='pink'>
Variables: {f o u r t w x x x}, generally the letters<br>
Domains: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}<br>
Constraints: 1. all letters have different values,<br> 2. addition works as intended(o + o = r + 10*x),<br> 3. leading digits aren't zero.
</font>

## Input
  - The first line is $N$ ,base of the sum operation. $2 \le N \le 36$. implementing CSP for $N$ other than 10 is optional and test cases with $N$ more than 10 has bonus score since it has a bigger search dimension so AC3, MRV and LCV is suggested.
  - In the following lines until the last, operands for the sum operation is given.
  - In the last line, $result$, the sum of strings is given.
  - **Note**: all the operands and the result are in lowercase.
 
$$1 \le N \le 50$$

### Sample Input
This sample describes the previous example.
```
10
two
two
four
```

## Output
for each given charachter as input, print it's corresponding number:
  - **Note**: order is not important. there are multiple solutions for this problem. if all the constraints are satisfied, you can return any solution.

### Sample Output
The solution to the example above is:
```
f 1
o 4
r 8
t 7
u 6
w 3
```

## Your code
**Note:** It's OK to change the signature for the given functions and the given structure is just a suggestion to help you with the implementation. If you tried to use another structure, the total score of that part won't be changed.

### Define Constraint (20 points)

In this part, write the satisfaction part of the constraint of this problem.
You can add any argument you want to `__init__` function.


- Check if all letters have a different value. (Unique Constraint)
- Check if the first letter of each word is not 0. (Not Zero Constraint)
- Check if the sum of the operands with assigned values is equal to the result. (Sum Constraint)
    - **Note**: if there are unassigned values for variables, you can ignore them and return true. for large inputs, you can use AC3 or MRV, LCV to reduce the search space with partial assignment.

In [2]:
import numpy as np


class Constraint:
    """
    Base class for all constraints.
    """

    def __init__(self, variable: str) -> None:
        """
        Initializes the constraint.

        Args:
            variable: name of the variable
        """
        self.variable = variable

    def satisfied(self, assignment: dict[str, int]) -> bool:
        """
        Checks if the constraint is satisfied.

        Args:
            assignment: dictionary of the assignment of the variables to the values

        Returns:
            False if the constraint is not satisfied, True otherwise (including if the variable is not assigned)
        """
        return True

class NotZeroConstraint(Constraint):
        """"
        Constraint that checks if the variable is not zero.
        """

        def satisfied(self, assignment: dict[str, int]) -> bool:
            if self.variable not in assignment:
                return True
            return assignment[self.variable] != 0

class UniqueConstraint(Constraint):
        """
        Constraint that checks if all the variables has unique values.
        """

        def __init__(self) -> None:
            pass

        def satisfied(self, assignment: dict[str, int]) -> bool:
            return len(assignment.keys()) == len(set(assignment.values()))

class SumConstraint(Constraint):
        """
        Constraint that checks if the sum of the operands is equal to the result.
        """

        def __init__(self, operands: list[str], result: str, base: int = 10, allow_carry: bool = False) -> None:
            """
            initializes the constraint.

            Args:
                operands: list of the operands
                result: result of the sum
                base: base of the numbers
                allow_carry: if true, the variables in the result is checked from right to left so sum of the operands can have a carry,
                             it is useful for partial sums of columns in AC3 or backtracking. implementation of this part is not required.
            """
            self.operands = operands
            self.result = result
            self.base = base
            self.allow_carry = allow_carry

        def satisfied(self, assignment: dict[str, int]) -> bool:
            # TODO: implement this method
            sum = 0
            carry = 0
            
            for i in range(len(result)):
                for operand in operands:
                    j = len(operand)
                    help_operand = operand[j - i - 1]
                    if j > i and help_operand not in assignment.keys():
                        return True
                    elif j > i:
                        sum += assignment[help_operand]
                        
                index = result[len(result) - i - 1]
                if index in assignment.keys():
                    carry += assignment[index] 
                else:
                    return True

                help_sum = sum - ((sum // base) * base)
                help_carry = carry - ((carry // base) * base)
                
                if help_sum == help_carry:
                    sum = sum // base
                    carry = carry // base 
                else:
                    return False
                
            return True    

### Define a CSP class (50 points)

You can add functions you'll need to this class.

In [3]:
class CSP:
    """
    CSP class that contains the variables, constraints and base of the problem.
    """

    def __init__(self, variables: list[str], constraints: list[Constraint], domains, base: int = 10) -> None:
        self.variables = variables
        self.constraints = constraints
        self.domains = domains
        self.base = base

    def consistent(self, assignment: dict[str, int], variable, value) -> bool:
        """
        Checks if the assignment is consistent with the constraints.

        Args:
            assignment: dictionary of the assignment of the variables to the values.

        Returns:
            False if the assignment is not consistent, True otherwise
        """
        new_assignment = assignment.copy()
        new_assignment[variable] = value
        for constraint in self.constraints:
            if not constraint.satisfied(new_assignment):
                return False
        return True

    def backtracking_search(self, assignment):
        """
        Backtracking search algorithm.

        Args:
            assignment: dictionary of the assignment of the variables to the values. assignment is modified in place.

        Returns:
            True if a solution is found, False otherwise
        """
        # TODO: implement this method
        var = self.select(assignment)
        if var == -1:
            return assignment
        new_assignment = assignment.copy()
        for value in self.domains[var]:
            if self.consistent(new_assignment, var, value):
                new_assignment[var] = value
                res = self.backtracking_search(new_assignment)
                if res != False:
                    return res
                del new_assignment[var]
        return False

    def select(self, assignment):

        for i in self.variables:
            if i not in assignment:
                return i
        return -1

### Build CSP and solve it! (20 points)

complete these functions for testing.

In [4]:
def get_problem(operands: list[str], result: str, base: int = 10) -> CSP:
    """
    Creates a CSP problem from the operands and the result. The variables are the unique characters in the operands and the result.
    The constraints should be defined usinng NotZeroConstraint for leftmost digits, a UniqueConstraint for all variables and a SumConstraint for the problem
    multiple partial SumConstraint can be used for bigger search spaces and base > 10.

    Args:
        operands: list of the operands
        result: result of the sum
        base: base of the numbers

    Returns:
        CSP problem
    """
    # TODO: implement this method
    variables = []
    for t in result:
        if t not in variables:
            variables.append(t)

    for i in range(len(operands)):
        for t in operands[i]:
            if t not in variables:
                variables.append(t)

    domains = {}
    for t in variables:
        domains[t] = [i for i in range(base)]

    c1 = NotZeroConstraint(result[0])
    cons_zero_list = []
    for i in range(len(operands)):
        C_i = NotZeroConstraint(operands[i][0])
        cons_zero_list.append(C_i)
    c2 = SumConstraint(operands, result, base)
    c3 = UniqueConstraint()
    constraints = list((c1, c2, c3))
    constraints.extend(cons_zero_list)
    return CSP(variables, constraints, domains, base)

In [5]:
def backtracking_search(csp: CSP) -> dict[str, int]:
    """
    Backtracking search algorithm.

    Args:
        csp: CSP problem
    
    Returns:
        dictionary of the assignment of the variables to the values if a solution is found, None otherwise
    """
    sol = {}
    ok = csp.backtracking_search(sol)
    if ok is not False:               
        return ok                 # changing the code
    return None


## Test

**Do not change the following cell.**

**Note**: 5 tests will be added after deadline. (5 points).

In [6]:
import helper.test_q2 as q2
import time

TIME_LIMIT = 3

tests = q2.get_all_tests(prefix='q2_')
tests_passed = 0
for test in tests:
    base, operands, result = q2.scan_test_input(test)
    csp = get_problem(operands, result, base)
    start_time = time.time()
    sol = backtracking_search(csp)
    if sol is None:
        print(f'test {test} failed. time elapsed={time.time() - start_time:.5f}s')
        continue
    total_time = time.time() - start_time
    if q2.is_result_valid(sol, operands, result, base) and total_time < TIME_LIMIT:
        tests_passed += 1
        print(f'test {test} passed. time elapsed={total_time:.5f}s')
        print('solution: ')
        print('\n'.join([str(x).upper() + " " + str(y)
              for x, y in sorted(sol.items())]))
    else:
        print(f'test {test} failed. time elapsed={total_time:.5f}s')
print(f'Score = {tests_passed / len(tests) * 100}%')


test q2_in1.txt passed. time elapsed=0.00000s
solution: 
X 1
Y 2
Z 8
test q2_in2.txt passed. time elapsed=0.00000s
solution: 
A 2
B 3
C 4
D 1
E 0
test q2_in3.txt passed. time elapsed=0.00097s
solution: 
A 1
B 9
C 8
test q2_in4.txt passed. time elapsed=0.00119s
solution: 
A 1
F 4
L 0
W 2
Y 5
test q2_in5.txt passed. time elapsed=0.00000s
solution: 
G 8
O 1
T 2
U 0
Score = 100.0%
